### Tips from prof

- Narrow scope of work (e.g. court level)

- Could try both binary/multi-class model outcomes and compare the performance 

- Change user from layperson to legal professional (and mention that this project is a stepping stone towards having layperson use the model)

- Link features to predicted outcome (if time permits can try using XGBoost with LIME for model interpretability)

- Can also try to see accuracy of models with different areas of law, lowest accuracy may be hardest area of law to predict


### Data setup

In [160]:
import os
import re
import spacy 
from spacy import displacy
import json
import gensim
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel, LsiModel, HdpModel
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
import ast
import nltk
import pyLDAvis
import pyLDAvis.gensim_models

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\benhz\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [161]:
# Load CSV files into DataFrames
areas_of_law_df = pd.read_csv("data/prediction_data/areas_of_law.csv", index_col=0)
coram_df = pd.read_csv("data/prediction_data/coram.csv", index_col=0)
sg_legal_cases_df = pd.read_csv("data/prediction_data/sg_legal_cases_dataset.csv", index_col=0)
target_rulings_df = pd.read_csv("data/prediction_data/target_rulings.csv", index_col=0)
# Load the JSON file into a dictionary
with open('data/prediction_data/issues.json') as f:
    issues_data = [json.loads(line) for line in f]
processed_df = pd.DataFrame(issues_data)

# Load the JSON file into a dictionary
with open('data/rawish_data/facts.json') as f:
    facts_data = [json.loads(line) for line in f]
raw_facts_df = pd.DataFrame(facts_data)

# Merge DataFrames
merged_df = pd.merge(areas_of_law_df, sg_legal_cases_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, processed_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, raw_facts_df, on='casename', how='outer')
merged_df = pd.merge(merged_df, target_rulings_df, on='casename', how='outer')
merged_df = merged_df.reset_index(drop=True)
try:
    merged_df.drop(columns=['Unnamed: 0'], inplace=True)
except:
    pass
# Display the resulting DataFrame
print(merged_df.head())

           casename                                        area_of_law  \
0   2000_SGCA_1.pdf  {'civil procedure': ['pleadings'], 'res judica...   
1  2000_SGCA_10.pdf  {'contract': ['formation'], 'equity': ['defenc...   
2  2000_SGCA_11.pdf  {'contract': ['discharge'], 'damages': ['asses...   
3  2000_SGCA_12.pdf  {'courts and jurisdiction': ['court of appeal'...   
4  2000_SGCA_13.pdf                     {'criminal law': ['offences']}   

  court_level                                             issues  \
0        SGCA  The claim was dismissed with costs by the\nHig...   
1        SGCA  the claim and\nagainst that decision this appe...   
2        SGCA  The appeal \nThe questions which arise in this...   
3        SGCA  the appeals from the assistant registrar. In h...   
4        SGCA  the appeal on 24 January 2000 and dismissed it...   

                                               facts        target  
0  The facts\nThe appellant is the widow of one T...    Favourable  
1  facts

In [162]:
merged_df = merged_df.dropna()
merged_df.isna().sum()

casename       0
area_of_law    0
court_level    0
issues         0
facts          0
target         0
dtype: int64

### Data Preprocessing

Remove duplicate coram names and roles

In [163]:
def clean_coram_names(coram_list):
    all_names = set()
    for item in coram_list:
        split_names = re.split(r';\s(?![a-zA-Z]+\s)', item)
        for name in split_names:
            if ';' in name and not re.search(r';\s[a-zA-Z]+$', name):
                sub_names = name.split(';')
                all_names.update([n.strip() for n in sub_names if n.strip()])
            else:
                all_names.add(name.strip())
    return list(all_names)

def remove_coram_roles(coram_list):
    roles = [' CJ', ' AG', ' J', ' DCJ', ' JA', ' AR', ' JC', 'SAR']
    for role in roles:
        coram_list = [re.sub(rf'{role}$', '', name) for name in coram_list]
    return coram_list

In [164]:
coram_df = coram_df.dropna()
for i, coram_str in enumerate(coram_df['Coram']):
    coram = ast.literal_eval(coram_str)
    
    coram_modified = clean_coram_names(coram)
    coram_modified = remove_coram_roles(coram_modified)
    coram_df.at[i, 'Coram'] = str(coram_modified)
merged_df = pd.merge(merged_df, coram_df, on='casename', how='outer')

In [165]:
nan_counts = merged_df.isna().sum()
print(nan_counts)

#nas are probably those reassigned cases, coram has 7, i just drop them for now
na_target_rows = merged_df[merged_df['target'].isna()]
print(na_target_rows)

merged_df.dropna(axis=0, inplace=True)
print(merged_df.isna().sum())

#remove empty lists
merged_df = merged_df.query("area_of_law != '[]'")

#target is unbalanced
target_counts = merged_df['target'].value_counts()
print(target_counts)

merged_df = merged_df.reset_index(drop=True) # prevent nan values from appearing after one-hot

casename        7
area_of_law    54
court_level    54
issues         54
facts          54
target         54
Coram           7
dtype: int64
               casename area_of_law court_level issues facts target  \
241   2000_SGHC_257.pdf         NaN         NaN    NaN   NaN    NaN   
274   2000_SGHC_290.pdf         NaN         NaN    NaN   NaN    NaN   
412    2001_SGCA_66.pdf         NaN         NaN    NaN   NaN    NaN   
432   2001_SGHC_101.pdf         NaN         NaN    NaN   NaN    NaN   
438   2001_SGHC_108.pdf         NaN         NaN    NaN   NaN    NaN   
442   2001_SGHC_111.pdf         NaN         NaN    NaN   NaN    NaN   
448   2001_SGHC_118.pdf         NaN         NaN    NaN   NaN    NaN   
457   2001_SGHC_128.pdf         NaN         NaN    NaN   NaN    NaN   
460   2001_SGHC_130.pdf         NaN         NaN    NaN   NaN    NaN   
462   2001_SGHC_132.pdf         NaN         NaN    NaN   NaN    NaN   
475   2001_SGHC_148.pdf         NaN         NaN    NaN   NaN    NaN   
478   200

In [166]:
merged_df['area_of_law'] = merged_df['area_of_law'].apply(ast.literal_eval)
merged_df['Coram'] = merged_df['Coram'].apply(ast.literal_eval)
merged_df.head(3)

,casename,area_of_law,court_level,issues,facts,target,Coram
0,2000_SGCA_1.pdf,"{'civil procedure': ['pleadings'], 'res judica...",SGCA,The claim was dismissed with costs by the\nHig...,The facts\nThe appellant is the widow of one T...,Favourable,"[V K Rajah, Chan Sek Keong, Andrew Phang Boon ..."
1,2000_SGCA_10.pdf,"{'contract': ['formation'], 'equity': ['defenc...",SGCA,the claim and\nagainst that decision this appe...,facts and surrounding circumstances including ...,Favourable,"[V K Rajah, Chao Hick Tin, Andrew Phang Boon L..."
2,2000_SGCA_11.pdf,"{'contract': ['discharge'], 'damages': ['asses...",SGCA,The appeal \nThe questions which arise in this...,"Background \nThe first appellants, a French co...",No outcome,"[Tan Lee Meng, Chan Sek Keong, Andrew Phang Bo..."


##### Flatten areas_of_law

Using Flattened AOL

In [167]:
def clean_list(law_list):
    cleaned_list = [item.replace(']', '').strip() for item in law_list]  # Removes ']' and extra spaces
    return cleaned_list

In [168]:
def flatten_keys_only(d):
    if isinstance(d, dict):
        return list(d.keys())
    return [] 

In [169]:
def flatten_areas(areas):
    flat_areas = []
    for main_area, sub_areas in areas.items():
        flat_areas.append(main_area)
        # Remove sub-area if it's longer than 33 characters
        sub_areas = [sarea for sarea in sub_areas if len(sarea) <= 33]
        flat_areas.extend(sub_areas)
    return flat_areas

In [170]:
merged_df['area_of_law'] = merged_df['area_of_law'].apply(flatten_keys_only)
# merged_df['area_of_law'] = merged_df['area_of_law'].apply(flatten_areas)
merged_df['area_of_law'] = merged_df['area_of_law'].apply(clean_list)

In [171]:
merged_df

,casename,area_of_law,court_level,issues,facts,target,Coram
0,2000_SGCA_1.pdf,"[civil procedure, res judicata, trusts]",SGCA,The claim was dismissed with costs by the\nHig...,The facts\nThe appellant is the widow of one T...,Favourable,"[V K Rajah, Chan Sek Keong, Andrew Phang Boon ..."
1,2000_SGCA_10.pdf,"[contract, equity]",SGCA,the claim and\nagainst that decision this appe...,facts and surrounding circumstances including ...,Favourable,"[V K Rajah, Chao Hick Tin, Andrew Phang Boon L..."
2,2000_SGCA_11.pdf,"[contract, damages]",SGCA,The appeal \nThe questions which arise in this...,"Background \nThe first appellants, a French co...",No outcome,"[Tan Lee Meng, Chan Sek Keong, Andrew Phang Bo..."
3,2000_SGCA_12.pdf,"[courts and jurisdiction, words and phrases]",SGCA,the appeals from the assistant registrar. In h...,"Background\nMicrosoft, Adobe and Autodesk are ...",Unfavourable,[Teo Guan Siew]
4,2000_SGCA_13.pdf,[criminal law],SGCA,the appeal on 24 January 2000 and dismissed it...,facts. Mere assertion would not suffice. In ex...,Unfavourable,[Lionel Yee]
...,...,...,...,...,...,...,...
6786,2023_SGHC_95.pdf,[criminal law],SGHC,the issues (and sub-issue) that arise for my \...,"Facts \n1 The first accused, Low Sze Song (“Lo...",No outcome,[Chan Seng Onn]
6787,2023_SGHC_96.pdf,[tort],SGHC,"the claim for conspiracy to defraud). At most,...","the facts, and can only be giving his opinion,...",Unfavourable,"[Tay Yong Kwang, Judith Prakash, Sundaresh Men..."
6788,2023_SGHC_97.pdf,[civil procedure],SGHC,the issues relating to the defences of \njusti...,"Facts\nThe parties \n3 The plaintiff, Mr Karan...",Unfavourable,[Woo Bih Li]
6789,2023_SGHC_98.pdf,[insolvency law],SGHC,,,Favourable,[Judith Prakash]


Using main area ONLY

One-hot Encoding

In [172]:
# one-hot encode aol
processed_df = merged_df[merged_df['area_of_law'].apply(lambda x: isinstance(x, list))]
processed_df = processed_df.reset_index(drop=True)
mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['area_of_law'])
binary_aol_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_aol_df = binary_aol_df.reset_index(drop=True)
processed_df = pd.concat([processed_df.drop('area_of_law', axis=1), binary_aol_df], axis=1)

print(processed_df.head(3))

           casename court_level  \
0   2000_SGCA_1.pdf        SGCA   
1  2000_SGCA_10.pdf        SGCA   
2  2000_SGCA_11.pdf        SGCA   

                                              issues  \
0  The claim was dismissed with costs by the\nHig...   
1  the claim and\nagainst that decision this appe...   
2  The appeal \nThe questions which arise in this...   

                                               facts      target  \
0  The facts\nThe appellant is the widow of one T...  Favourable   
1  facts and surrounding circumstances including ...  Favourable   
2  Background \nThe first appellants, a French co...  No outcome   

                                               Coram  \
0  [V K Rajah, Chan Sek Keong, Andrew Phang Boon ...   
1  [V K Rajah, Chao Hick Tin, Andrew Phang Boon L...   
2  [Tan Lee Meng, Chan Sek Keong, Andrew Phang Bo...   

   29 of the dissenting opinion) that  abitration  abuse of process  \
0                                   0           0                

In [173]:
# one-hot encode coram
processed_df = processed_df[processed_df['Coram'].apply(lambda x: isinstance(x, list))]

mlb = MultiLabelBinarizer()
binary_features = mlb.fit_transform(processed_df['Coram'])
binary_coram_df = pd.DataFrame(binary_features, columns=mlb.classes_)
binary_coram_df = binary_coram_df.reset_index(drop=True)
processed_df = pd.concat([processed_df.drop('Coram', axis=1), binary_coram_df], axis=1)

print(processed_df.head())

           casename court_level  \
0   2000_SGCA_1.pdf        SGCA   
1  2000_SGCA_10.pdf        SGCA   
2  2000_SGCA_11.pdf        SGCA   
3  2000_SGCA_12.pdf        SGCA   
4  2000_SGCA_13.pdf        SGCA   

                                              issues  \
0  The claim was dismissed with costs by the\nHig...   
1  the claim and\nagainst that decision this appe...   
2  The appeal \nThe questions which arise in this...   
3  the appeals from the assistant registrar. In h...   
4  the appeal on 24 January 2000 and dismissed it...   

                                               facts        target  \
0  The facts\nThe appellant is the widow of one T...    Favourable   
1  facts and surrounding circumstances including ...    Favourable   
2  Background \nThe first appellants, a French co...    No outcome   
3  Background\nMicrosoft, Adobe and Autodesk are ...  Unfavourable   
4  facts. Mere assertion would not suffice. In ex...  Unfavourable   

   29 of the dissenting opinion

In [174]:
processed_df['SGCA'] = processed_df['court_level'].apply(lambda x: 1 if x == 'SGCA' else 0)
processed_df['SGHC'] = processed_df['court_level'].apply(lambda x: 1 if x == 'SGHC' else 0)
processed_df = processed_df.drop('court_level', axis=1)

### Feature Engineering

#### Topic Modelling

Helper functions for Topic Modelling 

In [175]:
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[0-9]', '', text)
    text = re.sub(r'\W*\b(?!no)\w{1,2}\b', '', text)
    stop_words = set(stopwords.words('english'))
    legal_stopwords = ('appellant', 'respondent', 'plaintiff', 'defendant', 'mr', 'mrs', 'dr', 'mdm', 'court','version', 'hr', 'would', 'case', 'sghc', 'court', 'sgca', 'slr', 'sgdc', 'also', 'first', 'person', 'statement', 'line', 'para', 'fact', 'one', 'may', 'time', 'could', 'next', 'legal', 'issues', 'issue')
    stop_words.update(legal_stopwords)
    lemmatizer = WordNetLemmatizer()
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    
    return words

Topic Modelling For Facts

In [176]:
data = []

processed_df['processed_facts'] = processed_df['facts'].apply(preprocess_text)
processed_df.drop(columns=['facts'], inplace=True)
print(processed_df["processed_facts"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for fact in processed_df['processed_facts']:
    for word in fact:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# print(lda_model.show_topics())

# best_coherence = -1
# best_lda = None
# for num_topics in range(5, 31, 5):
#     # Train LDA model
#     lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                                 id2word=dictionary,
#                                                 num_topics=num_topics,
#                                                 random_state=42)
    
#     # Compute coherence score
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model_lda.get_coherence()
    
#     print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
#     if coherence_score > best_coherence:
#         best_coherence = coherence_score
#         best_topic = num_topics
# print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 10 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=10,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)

processed_df['facts_topic'] = topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [fact, widow, tan, geok, tee, deceased, sue, c...
1       [fact, surrounding, circumstance, including, a...
2       [background, appellant, french, company, secon...
3       [background, microsoft, adobe, autodesk, compa...
4       [fact, mere, assertion, suffice, exh, said, st...
                              ...                        
6786    [fact, accused, low, sze, song, low, year, old...
6787    [fact, giving, opinion, representation, amount...
6788    [fact, party, karan, bagga, litigant, proceedi...
6789                                                   []
6790    [fact, party, towa, company, incorporated, jap...
Name: processed_facts, Length: 6791, dtype: object
Finished preprocessing text
Performing topic modelling


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.100994  0.048446       1        1  19.403415
9      0.078567 -0.003466       2        1  14.594470
0      0.076891 -0.018260       3        1  11.846712
3     -0.096138  0.023140       4        1  11.355034
5     -0.085546 -0.007196       5        1   9.821182
6     -0.063103 -0.060811       6        1   8.749629
1     -0.015531  0.084308       7        1   8.500486
2      0.072321 -0.061915       8        1   6.372919
4     -0.006081  0.001378       9        1   4.704522
7     -0.062375 -0.005625      10        1   4.651630, topic_info=          Term          Freq         Total Category  logprob  loglift
812    accused  23333.000000  23333.000000  Default  30.0000  30.0000
1737  property  12397.000000  12397.000000  Default  29.0000  29.0000
3487      drug   5999.000000   5999.000000  Default  28.0000  28.0000
587        ltd  20831.000000  20831.000000  Default  27.0000  27.0000
1010       pte  16173.000000  16173.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
701    account    503.104732  14164.031820  Topic10  -5.8928  -0.2697
214    however    464.292694   7996.475282  Topic10  -5.9731   0.2217
39        made    446.291033  12244.941395  Topic10  -6.0126  -0.2439
25     defence    433.215461   7857.356543  Topic10  -6.0424   0.1700
132      dated    416.673835  11681.161974  Topic10  -6.0813  -0.2655

[864 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
31080      5  0.987696        aa
5898       1  0.007081   abdomen
5898       2  0.014161   abdomen
5898       3  0.007081   abdomen
5898       4  0.021242   abdomen
...      ...       ...       ...
13213      4  0.971656    yusoff
13213      5  0.004243    yusoff
13213      7  0.004243    yusoff
13213     10  0.004243    yusoff
38501      4  0.993517  zainudin

[4449 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 10, 1, 4, 6, 7, 2, 3, 5, 8])

In [177]:
print(processed_df.isna().sum())

casename                              0
issues                                0
target                                0
29 of the dissenting opinion) that    0
abitration                            0
                                     ..
Zhuo Wenzhao                          0
SGCA                                  0
SGHC                                  0
processed_facts                       0
facts_topic                           0
Length: 199, dtype: int64


#### Topic Modelling For Issues

In [178]:
data = []

processed_df['processed_issues'] = processed_df['issues'].apply(preprocess_text)
processed_df.drop(columns=['issues'], inplace=True)
print(processed_df["processed_issues"])
print("Finished preprocessing text")

texts, article = [], []
print("Performing topic modelling")
for issue in processed_df['processed_issues']:
    for word in issue:
        article.append(word)
        
    texts.append(article)
    article = []

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
# print(lda_model.show_topics())

# best_coherence = -1
# best_lda = None
# for num_topics in range(5, 31, 5):
#     # Train LDA model
#     lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
#                                                 id2word=dictionary,
#                                                 num_topics=num_topics,
#                                                 random_state=42)
#     # Compute coherence score
#     coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
#     coherence_score = coherence_model_lda.get_coherence()
    
#     print(f"Number of topics: {num_topics}, Coherence Score: {coherence_score}")
#     if coherence_score > best_coherence:
#         best_coherence = coherence_score
#         best_topic = num_topics
# print(f"Best no of topic: {best_topic} and Best Coherence Score: {best_coherence}")

#use the best model (result from above codes: 25 topics)
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                                id2word=dictionary,
                                                num_topics=25,
                                                random_state=42)

#inspiration from https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/
topics_matrix = lda_model[corpus]
topics = []
# Iterate over each document's topic distribution
# Get the topic with the highest probability
for doc in topics_matrix:
    topic = max(doc, key=lambda x: x[1])[0]
    topics.append(topic)
    
processed_df['issues_topic'] = topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

0       [claim, dismissed, cost, high, decision, fook,...
1       [claim, decision, appeal, brought, background,...
2       [appeal, question, arise, appeal, follows, app...
3       [appeal, assistant, registrar, ground, judgmen...
4       [appeal, january, dismissed, give, reason, evi...
                              ...                        
6786    [sub, arise, consideration, whether, low, siva...
6787    [claim, conspiracy, defraud, fault, ken, sally...
6788    [relating, defence, justification, qualified, ...
6789                                                   []
6790    [background, dispute, towa, commenced, suit, a...
Name: processed_issues, Length: 6791, dtype: object
Finished preprocessing text
Performing topic modelling


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.059610 -0.070087       1        1  8.459282
22     0.139164  0.012248       2        1  8.446789
9     -0.024383  0.050162       3        1  6.495624
2     -0.057170  0.036805       4        1  5.941991
17    -0.049759 -0.036058       5        1  4.928883
15    -0.015183  0.093140       6        1  4.795168
19    -0.039249 -0.021592       7        1  4.766215
8     -0.059233 -0.025101       8        1  4.647303
18     0.031058 -0.045772       9        1  4.412436
7     -0.002185 -0.109363      10        1  4.145175
14    -0.026493 -0.000311      11        1  3.722884
16    -0.038042  0.055699      12        1  3.594379
5     -0.000577  0.012677      13        1  3.571884
23    -0.037308  0.034388      14        1  3.421422
20     0.058433  0.051614      15        1  3.399946
13    -0.006485 -0.058857      16        1  3.178823
24     0.120575 -0.051383      17        1  3.104028
11    -0.016567  0.001603      18        1  3.058579
0     -0.044258  0.022582      19        1  3.024890
1      0.097835  0.022025      20        1  2.749443
6      0.055858  0.014277      21        1  2.744105
4     -0.018616  0.008079      22        1  2.299619
12    -0.005027 -0.015865      23        1  2.282377
21    -0.012179 -0.003553      24        1  2.025223
10     0.009403  0.022643      25        1  0.783529, topic_info=             Term           Freq          Total Category  logprob  loglift
1151      accused   39061.000000   39061.000000  Default  30.0000  30.0000
3697         drug   17606.000000   17606.000000  Default  29.0000  29.0000
2962  arbitration   22786.000000   22786.000000  Default  28.0000  28.0000
2035         mark   16221.000000   16221.000000  Default  27.0000  27.0000
641           ltd  128167.000000  128167.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
730         order     526.682801   75977.647069  Topic25  -5.7335  -0.1225
70         appeal     522.585574   75173.646408  Topic25  -5.7414  -0.1196
286      decision     495.870998   54891.120675  Topic25  -5.7938   0.1423
191         claim     526.459458  101091.362193  Topic25  -5.7340  -0.4085
245      contract     489.017893   57295.351536  Topic25  -5.8077   0.0855

[2169 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
24874      1  0.041061      aa
24874      3  0.005475      aa
24874      4  0.035586      aa
24874      7  0.005475      aa
24874      9  0.019162      aa
...      ...       ...     ...
20726     19  0.029710  zoning
20726     21  0.029710  zoning
20726     22  0.014855  zoning
20726     24  0.519923  zoning
20726     25  0.014855  zoning

[21373 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 23, 10, 3, 18, 16, 20, 9, 19, 8, 15, 17, 6, 24, 21, 14, 25, 12, 1, 2, 7, 5, 13, 22, 11])

### Convolutional Neural Networks (CNN)

Train & Test set for CNN

In [179]:
X = processed_df.drop(columns=['target'])
y = processed_df['target']

stratified_split = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, remaining_index in stratified_split.split(X, y):
    X_train, X_test_val = X.iloc[train_index], X.iloc[remaining_index]
    y_train, y_test_val = y.iloc[train_index], y.iloc[remaining_index]

#balanced dataset (target variable was imbalanced Favourable 5006 Unfavourable 2523 No outcome 984)
#randomly found one online, can be changed -> need to check am i doing this right 
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

#split further from X_test_val into X_val and X_test
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=42, stratify=y_test_val)

Convert target variable to continuous

In [180]:
X_train_resampled = X_train_resampled.drop(columns=['processed_facts', 'processed_issues'])
X_test = X_test.drop(columns=['processed_facts', 'processed_issues'])
X_val = X_val.drop(columns=['processed_facts', 'processed_issues'])

mapping = {'Favourable': 1, 'Unfavourable': 0, 'No outcome':0.5}

y_train_resampled, y_test, y_val = y_train_resampled.copy().map(mapping), y_test.copy().map(mapping), y_val.copy().map(mapping)

#### Modeling (CNN)

In [181]:
# Perform modelling
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import tensorflow as tf

Hyper params

In [182]:
class Args:
  epochs = 20
  lr = 0.001
  use_cuda=False
  gamma = 0.7
  log_interval = 10
  seed = 1

args = Args()

device = torch.device("cuda" if args.use_cuda else "cpu")

In [183]:
X_train_resampled = X_train_resampled.iloc[:, 1:1596].copy()
X_train_resampled = torch.tensor(X_train_resampled.values, dtype=torch.float32).to(device)
print(f'Shape of X_train_resampled: {X_train_resampled.shape}')

X_test = X_test.iloc[:, 1:1596].copy()
X_test = torch.tensor(X_test.values, dtype=torch.float32).to(device)
print(f'Shape of X_test: {X_test.shape}')

X_val = X_val.iloc[:, 1:1596].copy()
X_val = torch.tensor(X_val.values, dtype=torch.float32).to(device)
print(f'Shape of X_val: {X_val.shape}')

Shape of X_train_resampled: torch.Size([8274, 196])
Shape of X_test: torch.Size([1019, 196])
Shape of X_val: torch.Size([1019, 196])


In [184]:
y_train_resampled, y_test, y_val = torch.tensor(y_train_resampled.values).to(device), torch.tensor(y_test.values).to(device), torch.tensor(y_val.values).to(device)

X_train_resampled = X_train_resampled.reshape(X_train_resampled.shape[0],1,X_train_resampled.shape[1])
X_test = X_test.reshape(X_test.shape[0],1,X_test.shape[1])
X_val = X_val.reshape(X_val.shape[0],1,X_val.shape[1])
print(X_train_resampled.shape)

torch.Size([8274, 1, 196])


#### Training CNN

In [187]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(1, 64, 3, 1,1, bias=True)
        # Define the first 1D convolution layer. Takes 1 input channel, outputs 32 channels, kernel size is 3, stride is 1, padding is 1.
        self.Bn1 = nn.BatchNorm1d(64)
        # Apply Batch Normalization to the output of the first convolutional layer.
        self.dropout = nn.Dropout(0.3)
        self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
        # Apply 1D Average Pooling after the first Batch Normalization. The kernel size and stride are 2.

        self.conv2 = nn.Conv1d(64, 64, 3, 1,1, bias=True)
        self.Bn2 = nn.BatchNorm1d(64)
        self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)

        self.fc1 = nn.Linear(3136, 100, bias=True)
        # Define the first fully connected layer. It takes 25472 inputs and outputs 100 nodes.

        self.fc2 = nn.Linear(100, 30, bias=True)
        # Define the second fully connected layer. It takes 100 inputs and outputs 50 nodes.

        self.fc3 = nn.Linear(30, 3, bias=True)
        # Define the third fully connected layer (output layer). It takes 50 inputs and outputs 3 nodes.


    def forward(self, x):
        x = F.relu(self.Bn1(self.conv1(x)))
        # Pass the input through the first convolutional layer, then Batch Normalization, and then apply ReLU activation.
        x = self.dropout(x)
        x = self.pool1(x)
        # Apply Average Pooling to the output of the previous step.
        x = F.relu(self.Bn2(self.conv2(x)))
        x = self.dropout(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        # Flatten the output from the previous step. This is necessary because fully connected layers expect a 1D input.
        x = self.fc1(x)
        # Pass the output through the first fully connected layer.
        x = F.relu(self.fc2(x))
        # Pass the output through the second fully connected layer with ReLU activation.
        x = self.fc3(x)
        # Pass the output through the third fully connected layer. This is the output of the network.
        return x
    
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()  # Set the model to training mode

    for batch_idx, (data, target) in enumerate(train_loader):  # Loop over each batch from the training set
        data, target = data.to(device), target.to(device)  # Move the data to the device that is used

        target = target.long()  # Make sure that target data is long type (necessary for loss function)

        optimizer.zero_grad()  # Clear gradients from the previous training step
        output = model(data)  # Run forward pass (model predictions)

        loss = F.cross_entropy(output, target)  # Calculate the loss between the output and target
        loss.backward()  # Perform backpropagation (calculate gradients of loss w.r.t. parameters)
        optimizer.step()  # Update the model parameters

        if batch_idx % args.log_interval == 0:  # Print log info for specified interval
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader):
    model.eval()  # Set the model to evaluation mode
    test_loss = 0
    correct = 0

    with torch.no_grad():  # Deactivates autograd, reduces memory usage and speeds up computations
        for data, target in test_loader:  # Loop over each batch from the testing set
            
            data, target = data.to(device), target.to(device)  # Move the data to the device that is used

            target = target.long()  # Convert target to long after adjusting value
            output = model(data)  # Run forward pass (model predictions)
            test_loss += F.cross_entropy(output, target, reduction='sum').item()  # Sum up the batch loss
            pred = output.argmax(dim=1, keepdim=True)  # Get the index of the max log-probability as the predicted output
            correct += pred.eq(target.view_as(pred)).sum().item()  # Count correct predictions

    test_loss /= len(test_loader.dataset)  # Calculate the average loss

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(test_loss, correct, len(test_loader.dataset),100. * correct / len(test_loader.dataset)))
    return correct  # Return the number of correctly classified samples


In [188]:
torch.manual_seed(args.seed)

model = Net().to(device)

for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())

#Form training and testing dataset
optimizer = optim.Adam(model.parameters(), lr=args.lr)

train_dataset = torch.utils.data.TensorDataset(X_train_resampled, y_train_resampled)
test_dataset = torch.utils.data.TensorDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
scheduler = StepLR(optimizer, step_size=1, gamma=args.gamma)

#Model training
ACC = 0
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    ACC_ = test(model, device, test_loader)
    if ACC_>ACC or ACC_ == ACC:
        ACC = ACC_
        torch.save(model.state_dict(), "Baseline_CNN.pt")

    scheduler.step()

print(ACC)


conv1.weight 	 torch.Size([64, 1, 3])
conv1.bias 	 torch.Size([64])
Bn1.weight 	 torch.Size([64])
Bn1.bias 	 torch.Size([64])
Bn1.running_mean 	 torch.Size([64])
Bn1.running_var 	 torch.Size([64])
Bn1.num_batches_tracked 	 torch.Size([])
conv2.weight 	 torch.Size([64, 64, 3])
conv2.bias 	 torch.Size([64])
Bn2.weight 	 torch.Size([64])
Bn2.bias 	 torch.Size([64])
Bn2.running_mean 	 torch.Size([64])
Bn2.running_var 	 torch.Size([64])
Bn2.num_batches_tracked 	 torch.Size([])
fc1.weight 	 torch.Size([100, 3136])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([30, 100])
fc2.bias 	 torch.Size([30])
fc3.weight 	 torch.Size([3, 30])
fc3.bias 	 torch.Size([3])
Train Epoch: 1 [0/8274 (0%)]	Loss: 1.082497
Train Epoch: 1 [640/8274 (8%)]	Loss: 0.642451
Train Epoch: 1 [1280/8274 (15%)]	Loss: 0.685211
Train Epoch: 1 [1920/8274 (23%)]	Loss: 0.679216
Train Epoch: 1 [2560/8274 (31%)]	Loss: 0.671502
Train Epoch: 1 [3200/8274 (38%)]	Loss: 0.723322
Train Epoch: 1 [3840/8274 (46%)]	Loss: 0.663082
Trai

### Random Forest (RF)

In [189]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV

#### Train & Test set for RF

In [190]:
X = processed_df.drop(['target', 'casename'], axis=1)
y = processed_df['target'] 

# Handle imbalanced classes
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Splitting the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

In [191]:
# Vectorizing textual features using TF-IDF for X_train
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X_train_text = tfidf_vectorizer.fit_transform(X_train['processed_facts'].astype('U') + ' ' + X_train['processed_issues'].astype('U'))
X_train_text = pd.DataFrame(X_train_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Drop original text columns and concatenate TF-IDF features
X_train = X_train.drop(['processed_facts', 'processed_issues'], axis=1)
X_train = pd.concat([X_train.reset_index(drop=True), X_train_text], axis=1)

# Vectorizing textual features using TF-IDF for X_test
X_test_text = tfidf_vectorizer.transform(X_test['processed_facts'].astype('U') + ' ' + X_test['processed_issues'].astype('U'))
X_test_text = pd.DataFrame(X_test_text.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# Drop original text columns and concatenate TF-IDF features
X_test = X_test.drop(['processed_facts', 'processed_issues'], axis=1)
X_test = pd.concat([X_test.reset_index(drop=True), X_test_text], axis=1)


KeyboardInterrupt: 

#### Modelling (RF)

Hyper Param Tuning (Grid search)

In [ ]:
rf_classifier = RandomForestClassifier(random_state=42)
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Setup the grid search
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, cv=5, scoring='accuracy', verbose=1, n_jobs=-1)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters and best score
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Optionally, use the best estimator to make predictions
best_rf = grid_search.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation score: 0.82


Model Training

In [ ]:
# Train the model using the training sets
best_rf.fit(X_train, y_train)

y_pred = best_rf.predict(X_test)

# Evaluating the Model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8486471251409244
Classification Report:
              precision    recall  f1-score   support

  Favourable       0.73      0.89      0.80      1196
  No outcome       0.98      0.97      0.98      1152
Unfavourable       0.87      0.69      0.77      1200

    accuracy                           0.85      3548
   macro avg       0.86      0.85      0.85      3548
weighted avg       0.86      0.85      0.85      3548



### Evaluation